In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from pylab import *
from pyntcloud import PyntCloud
from matplotlib.pyplot import figure

INFO - 2021-12-22 16:06:58,054 - utils - NumExpr defaulting to 8 threads.


## Please note that this notebook is designed for constructing a model from the raw PointCloud. Do not run this notebook on a already constructed model, which will impair the completeness and the accuracy of the model.

# Visualize Point Cloud

reads a point cloud model of Venice and visualizes it

In [2]:
print("Load a ply point cloud")
venice = o3d.io.read_point_cloud('/Users/liyuxiao/Downloads/venice_building_height/google_earth/google_earth.ply')

Load a ply point cloud


In [11]:
print("Print (and visualize) the ply point cloud")
print(venice)
print(np.asarray(venice.points))
o3d.visualization.draw_geometries([venice])

Print (and visualize) the ply point cloud
PointCloud with 4093480 points.
[[-3.21571106e+02 -2.48106494e+03  1.64116311e+00]
 [-3.28938873e+02 -2.48008789e+03  1.29477739e+00]
 [-3.28718689e+02 -2.47819043e+03  1.62962472e+00]
 ...
 [ 1.35218542e+03  1.88591583e+02  9.99854851e+00]
 [-7.04763123e+02  9.39715210e+02  7.02373171e+00]
 [-9.63983093e+02  3.89082733e+02  3.52930188e+00]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Voxel Downsampling
Voxel downsampling uses a regular voxel grid to create a uniformly downsampled point cloud from an input point cloud. It is often used as a pre-processing step for many point cloud processing tasks. The algorithm operates in two steps:
1. Points are bucketed into voxels.
2. Each occupied voxel generates exactly one point by averaging all points inside.

In [12]:
print("Downsample the point cloud with a voxel of 0.05")
venice_down = venice.voxel_down_sample(voxel_size = 0.05)

Downsample the point cloud with a voxel of 0.05


After we downsampled the PointCloud, it should be much easier to visualize the PointCloud

In [17]:
o3d.visualization.draw_geometries([venice_down])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Plane Segmentation
Open3D also supports segmententation of geometric primitives from point clouds using RANSAC. To find the plane with the largest support in the point cloud, we can use segment_plane. The method has three arguments: distance_threshold defines the maximum distance a point can have to an estimated plane to be considered an inlier, ransac_n defines the number of points that are randomly sampled to estimate a plane, and num_iterations defines how often a random plane is sampled and verified. The function then returns the plane as (𝑎,𝑏,𝑐,𝑑) such that for each point (𝑥,𝑦,𝑧) on the plane we have 𝑎𝑥+𝑏𝑦+𝑐𝑧+𝑑=0. The function further returns a list of indices of the inlier points.

In [4]:
plane_model, inliers = venice_down.segment_plane(distance_threshold=0.01,
                                         ransac_n=5,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a}x + {b}y + {c}z + {d} = 0")

Plane equation: 9.751275202931665e-05x + 0.0007174054486077785y + 0.9999997379103084z + 0.11320834757526194 = 0


Visualize the plane

In [15]:
inlier_cloud = venice_down.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = venice_down.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Point Cloud Outlier Removal
statistical_outlier_removal removes points that are further away from their neighbors compared to the average for the point cloud. It takes two input parameters:

1. nb_neighbors, which specifies how many neighbors are taken into account in order to calculate the average distance for a given point.
2. std_ratio, which allows setting the threshold level based on the standard deviation of the average distances across the point cloud. The lower this number the more aggressive the filter will be.

In [16]:
print("Statistical oulier removal")
venice_down, ind = venice_down.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

Statistical oulier removal
Remove statistical outliers: [========================================] 100%


# Point Cloud Redressing

Please note that here we need to do an iteration so that we could get a more precise model

1. We first rotate the PointCloud so that the plane in the PointCloud is XoY plane
2. Then we scale the PointCloud so that the height in the model would match the height in real world
3. In the end, we translate the PointCloud so that the plane equation could be Z=0

In [17]:
for i in range(1000):

    cos = c
    sin = np.sqrt(a**2+b**2)
    u1= b
    u2= -a

    R = [[cos+u1**2*(1-cos), u1*u2*(1-cos), u2*sin],
    [u1*u2*(1-cos),cos+u2**2*(1-cos),-u1*sin],
    [-u2*sin, u1*sin, cos]]
    
    venice_down.rotate(R, center=venice_down.get_center())

    venice_coordinate = np.asarray(venice_down.points)
    distance = np.zeros((len(venice_coordinate),1))
    for i in range(len(venice_coordinate)):
        distance[i] = abs(venice_coordinate[i,0]*a+ venice_coordinate[i,1]*b+ venice_coordinate[i,2]*c +d)/np.sqrt(a**2+b**2+c**2)

    venice_down.scale(98.6/distance[np.argmax(distance, axis=0)][0,0], center=venice_down.get_center())

    plane_model, inliers = venice_down.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    [a, b, c, d] = plane_model
    venice_down = venice_down.translate((0, 0, d))
    
    print(f"Plane equation: {a}x + {b}y + {c}z + {d} = 0")

Plane equation: 1.7319796234134454e-05x + 0.0003149600472801958y + 0.9999999502500954z + 3.5714228076475067 = 0
Plane equation: 0.003565504676291095x + -0.0006274814378795525y + 0.9999934467002513z + -0.3112692023052903 = 0
Plane equation: 0.0019504628118613546x + 0.0016724274426002034y + 0.9999966993351873z + 0.8060104421114023 = 0
Plane equation: 0.004343105385845667x + 0.0012683199492295944y + 0.9999897643476726z + -0.829525972328512 = 0
Plane equation: 0.0035333792543046708x + -0.0005125540880867755y + 0.9999936262393636z + 0.6204380176838785 = 0
Plane equation: 0.0056559357829633426x + -0.00024633851024181665y + 0.9999839747254741z + -0.8446119574231453 = 0
Plane equation: 0.0019691973419972714x + -0.0006627391122753552y + 0.9999978415170192z + 0.4812957797183621 = 0
Plane equation: -0.007957011244553147x + 0.004714509403682265y + 0.9999572287718793z + -0.6275770026512567 = 0
Plane equation: 0.0043162081227538255x + -0.0038424721252265604y + 0.9999833027383047z + 0.542779751291363

# Verification

In [5]:
venice_coordinate = np.asarray(venice_down.points)
distance = np.zeros((len(venice_coordinate),1))
for i in range(len(venice_coordinate)):
    distance[i] = abs(venice_coordinate[i,0]*a+ venice_coordinate[i,1]*b+ venice_coordinate[i,2]*c +d)/np.sqrt(a**2+b**2+c**2)

Z coordinate of the height point in Venice: (reference:98.6m)

In [6]:
venice_coordinate[np.argmax(distance, axis=0)][0,2]

98.48127746582031

Distance from the height ppoint to the ground: (reference:98.6m)

In [7]:
distance[np.argmax(distance, axis=0)][0,0]

98.30085345850753

Take a look at the PointCloud model you constructed

In [8]:
o3d.visualization.draw_geometries([venice_down])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Dada! Right now we finished the model construction part, we could save the model here.

In [70]:
o3d.io.write_point_cloud("/Users/liyuxiao/Downloads/venice_building_height/youtube1/youtube1_temp.ply", venice_down)

True